In [2]:
!pip install pandas
!pip install selenium


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [4]:
df_link_data = pd.read_csv('data/link_data_2.csv')
df_link_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  998 non-null    int64 
 1   link        998 non-null    object
dtypes: int64(1), object(1)
memory usage: 15.7+ KB


In [5]:
LIST_LINK_PRODUCT = df_link_data['link'].values.tolist()
print(len(LIST_LINK_PRODUCT))

998


In [6]:
LIST_PRODUCT = []

# Define a function to extract property details
def extract_property_details(url):
    property_details = {
        'type_estate': '',
        'district': '',
        'price_per_sqm': '',
        'posted_date': '',
        'area': '',
        'price': '',
        'legal_document': '',
        'interior': '',
        'num_bedrooms': '',
        'num_bathrooms': '',
        'num_floors': '',
        'house_orientation': '',
        'balcony_orientation': '',
        'entrance': '',
        'frontage': ''
    }

    try:
        # Set up Selenium WebDriver
        with webdriver.Chrome() as driver:
            driver.get(url)
            
            # Get the page source
            page_source = driver.page_source

        soup = BeautifulSoup(page_source, "html.parser")

        try:
            # Find the breadcrumb element
            breadcrumb = soup.find('div', class_='re__breadcrumb')
            if breadcrumb:
                level_4_link = breadcrumb.find('a', {'level': '4'})
                if level_4_link:
                    level_4_title = level_4_link.text

                    # Use regular expressions to extract the type_estate
                    match = re.search(r"(.+?) tại", level_4_title)
                    type_estate = match.group(1) if match else ""
                    property_details['type_estate'] = type_estate

                level_3_link = breadcrumb.find('a', {'level': '3'})
                if level_3_link:
                    property_details['district'] = level_3_link.text
            else:
                print(f"Breadcrumb not found for URL: {url}")
        except AttributeError as e:
            print(f"Error extracting type_estate and district: {e}")

        # Extract price per square meter
        try:
            price_per_sqm_span = soup.find('span', class_='ext')
            if price_per_sqm_span:
                property_details['price_per_sqm'] = price_per_sqm_span.text.strip()
        except AttributeError as e:
            print(f"Error extracting price_per_sqm: {e}")

        # Extract posted date
        try:
            posted_date_span = soup.find('span', string='Ngày đăng')
            if posted_date_span:
                property_details['posted_date'] = posted_date_span.find_next_sibling('span').text.strip()
        except AttributeError as e:
            print(f"Error extracting posted_date: {e}")

        # Extract property specifications
        try:
            specs_content = soup.find_all('div', class_='re__pr-specs-content-item')
            # Mapping of titles to corresponding keys in property_details
            title_map = {
                'Diện tích': 'area',
                'Mức giá': 'price',
                'Pháp lý': 'legal_document',
                'Nội thất': 'interior',
                'Số phòng ngủ': 'num_bedrooms',
                'Số toilet': 'num_bathrooms',
                'Số tầng': 'num_floors',
                'Hướng nhà': 'house_orientation',
                'Hướng ban công': 'balcony_orientation',
                'Đường vào': 'entrance',
                'Mặt tiền': 'frontage'
            }

            # Extract and map property details
            for item in specs_content:
                title = item.find(class_='re__pr-specs-content-item-title').text.strip()
                if title in title_map:
                    property_details[title_map[title]] = item.find(class_='re__pr-specs-content-item-value').text.strip()
        except AttributeError as e:
            print(f"Error extracting property details: {e}")
        
    except Exception as e:
        print(f"Failed to extract data from {url}: {e}")

    return property_details

def scrape_property_details(urls):
    scraped_properties = []
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(extract_property_details, url): url for url in urls}
        try:
            for future in as_completed(futures):
                url = futures[future]
                try:
                    result = future.result()
                    print(result)  # Print the extracted details
                    scraped_properties.append(result)
                except Exception as e:
                    print(f"Failed to extract data from {url}: {e}")
        except KeyboardInterrupt:
            for f in futures:
                f.cancel()
            raise

    return scraped_properties

LIST_PRODUCT = scrape_property_details(LIST_LINK_PRODUCT)
print(LIST_PRODUCT)

Breadcrumb not found for URL: https://batdongsan.com.vn/ban-nha-rieng-pho-xa-dan-phuong-nam-dong/ban-99m2-9tang-thang-may-29-phong-kep-kin-pr39841291
{'type_estate': '', 'district': '', 'price_per_sqm': '', 'posted_date': '', 'area': '', 'price': '', 'legal_document': '', 'interior': '', 'num_bedrooms': '', 'num_bathrooms': '', 'num_floors': '', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '', 'frontage': ''}
Breadcrumb not found for URL: https://batdongsan.com.vn/ban-nha-mat-pho-phuong-ngoc-lam-2/-90m2-long-bien-29-5ty-lo-goc-oto-tai-do-thang-may-o-kinh-doanh-dinh-thang-may-pr39841067
{'type_estate': '', 'district': '', 'price_per_sqm': '', 'posted_date': '', 'area': '', 'price': '', 'legal_document': '', 'interior': '', 'num_bedrooms': '', 'num_bathrooms': '', 'num_floors': '', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '', 'frontage': ''}
{'type_estate': 'Nhà riêng', 'district': 'Cầu Giấy', 'price_per_sqm': '~200 triệu/m²', 'posted_date': '05/

In [7]:
print(len(LIST_PRODUCT))

998


In [11]:
df_link_product = pd.DataFrame(LIST_PRODUCT)
df_link_product.to_csv('data/raw_data_2.csv', index=False)

In [15]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("data/raw_data.csv")

# Drop the 'link' and 'describe' columns
df.drop(['link', 'describe'], axis=1, inplace=True)

# Save the modified DataFrame back to a CSV file
df.to_csv("data/raw_data.csv", index=False)

KeyError: "['link', 'describe'] not found in axis"

In [10]:
df = pd.read_csv('data/raw_data_2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           998 non-null    int64 
 1   type_estate          884 non-null    object
 2   district             884 non-null    object
 3   price_per_sqm        816 non-null    object
 4   posted_date          883 non-null    object
 5   area                 883 non-null    object
 6   price                883 non-null    object
 7   legal_document       778 non-null    object
 8   interior             545 non-null    object
 9   num_bedrooms         643 non-null    object
 10  num_bathrooms        567 non-null    object
 11  num_floors           808 non-null    object
 12  house_orientation    132 non-null    object
 13  balcony_orientation  76 non-null     object
 14  entrance             406 non-null    object
 15  frontage             594 non-null    object
dtypes: int64